# 连表查询-定义一对多/多对一表


## 测试准备


### 定义 model


In [1]:
from typing import Optional

from sqlmodel import Field, SQLModel, create_engine


class Team(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(index=True)
    headquarters: str


class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(index=True)
    secret_name: str
    age: Optional[int] = Field(default=None, index=True)

    team_id: Optional[int] = Field(default=None, foreign_key="team.id")

### 创建表


In [8]:
sqlite_file_name = "join_database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"

engine = create_engine(sqlite_url, echo=False)

SQLModel.metadata.create_all(engine)
from sqlmodel import Session
from sqlmodel import select

### 添加测试数据


In [9]:
with Session(engine) as session:
    team_preventers = Team(name="Preventers", headquarters="Sharp Tower")
    team_z_force = Team(name="Z-Force", headquarters="Sister Margaret's Bar")
    session.add(team_preventers)
    session.add(team_z_force)
    session.commit()

    hero_deadpond = Hero(
        name="Deadpond", secret_name="Dive Wilson", team_id=team_z_force.id
    )
    hero_rusty_man = Hero(
        name="Rusty-Man",
        secret_name="Tommy Sharp",
        age=48,
        team_id=team_preventers.id,
    )
    hero_spider_boy = Hero(name="Spider-Boy", secret_name="Pedro Parqueador")

    session.add(hero_deadpond)
    session.add(hero_rusty_man)
    session.add(hero_spider_boy)
    session.commit()
    session.refresh(hero_deadpond)
    session.refresh(hero_rusty_man)
    session.refresh(hero_spider_boy)

    print("Created hero:", hero_deadpond)
    print("Created hero:", hero_rusty_man)
    print("Created hero:", hero_spider_boy)

Created hero: secret_name='Dive Wilson' name='Deadpond' team_id=4 age=None id=4
Created hero: secret_name='Tommy Sharp' name='Rusty-Man' team_id=3 age=48 id=5
Created hero: secret_name='Pedro Parqueador' name='Spider-Boy' team_id=None age=None id=6


## 连表方法

使用 where 和使用 join 的 sql 是不同的


### 方式一 where 连表


In [4]:
with Session(engine) as session:
    statement = select(Hero, Team).where(Hero.team_id == Team.id)
    results = session.exec(statement).all()
    for hero, team in results:
        print("Hero:", hero, "  Team:", team)

2023-12-29 15:20:21,770 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-29 15:20:21,773 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id, team.id AS id_1, team.name AS name_1, team.headquarters 
FROM hero, team 
WHERE hero.team_id = team.id
2023-12-29 15:20:21,774 INFO sqlalchemy.engine.Engine [generated in 0.00115s] ()
Hero: secret_name='Dive Wilson' name='Deadpond' team_id=2 age=None id=1   Team: name='Z-Force' headquarters="Sister Margaret's Bar" id=2
Hero: secret_name='Tommy Sharp' name='Rusty-Man' team_id=1 age=48 id=2   Team: name='Preventers' headquarters='Sharp Tower' id=1
2023-12-29 15:20:21,777 INFO sqlalchemy.engine.Engine ROLLBACK


### 方式二 join 连表


In [5]:
with Session(engine) as session:
    statement = select(Hero, Team).join(Team)
    results = session.exec(statement)
    for hero, team in results:
        print("Hero:", hero, "  Team:", team)

2023-12-29 15:20:28,001 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-29 15:20:28,004 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id, team.id AS id_1, team.name AS name_1, team.headquarters 
FROM hero JOIN team ON team.id = hero.team_id
2023-12-29 15:20:28,005 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ()
Hero: secret_name='Dive Wilson' name='Deadpond' team_id=2 age=None id=1   Team: name='Z-Force' headquarters="Sister Margaret's Bar" id=2
Hero: secret_name='Tommy Sharp' name='Rusty-Man' team_id=1 age=48 id=2   Team: name='Preventers' headquarters='Sharp Tower' id=1
2023-12-29 15:20:28,007 INFO sqlalchemy.engine.Engine ROLLBACK


## 连接方向


### inner


In [6]:
with Session(engine) as session:
    statement = select(Hero, Team).join(
        Team,
    )
    results = session.exec(statement)
    for hero, team in results:
        print("Hero:", hero, "Team:", team)

2023-12-29 15:20:42,591 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-29 15:20:42,593 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id, team.id AS id_1, team.name AS name_1, team.headquarters 
FROM hero JOIN team ON team.id = hero.team_id
2023-12-29 15:20:42,593 INFO sqlalchemy.engine.Engine [cached since 14.59s ago] ()
Hero: secret_name='Dive Wilson' name='Deadpond' team_id=2 age=None id=1 Team: name='Z-Force' headquarters="Sister Margaret's Bar" id=2
Hero: secret_name='Tommy Sharp' name='Rusty-Man' team_id=1 age=48 id=2 Team: name='Preventers' headquarters='Sharp Tower' id=1
2023-12-29 15:20:42,595 INFO sqlalchemy.engine.Engine ROLLBACK


### outer


In [7]:
with Session(engine) as session:
    statement = select(Hero, Team).join(Team, isouter=True)
    results = session.exec(statement)
    for hero, team in results:
        print("Hero:", hero, "Team:", team)

2023-12-29 15:25:07,727 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-29 15:25:07,729 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age, hero.team_id, team.id AS id_1, team.name AS name_1, team.headquarters 
FROM hero LEFT OUTER JOIN team ON team.id = hero.team_id
2023-12-29 15:25:07,731 INFO sqlalchemy.engine.Engine [generated in 0.00194s] ()
Hero: secret_name='Dive Wilson' name='Deadpond' team_id=2 age=None id=1 Team: name='Z-Force' headquarters="Sister Margaret's Bar" id=2
Hero: secret_name='Tommy Sharp' name='Rusty-Man' team_id=1 age=48 id=2 Team: name='Preventers' headquarters='Sharp Tower' id=1
Hero: secret_name='Pedro Parqueador' name='Spider-Boy' team_id=None age=None id=3 Team: None
2023-12-29 15:25:07,734 INFO sqlalchemy.engine.Engine ROLLBACK


## 连表查询


### join 和 where 连用


In [10]:
with Session(engine) as session:
    statement = select(Hero, Team).join(Team).where(Team.name == "Preventers")
    results = session.exec(statement)
    for hero, team in results:
        print("Preventer Hero:", hero, "Team:", team)

Preventer Hero: secret_name='Tommy Sharp' name='Rusty-Man' team_id=1 age=48 id=2 Team: name='Preventers' headquarters='Sharp Tower' id=1
Preventer Hero: secret_name='Tommy Sharp' name='Rusty-Man' team_id=3 age=48 id=5 Team: name='Preventers' headquarters='Sharp Tower' id=3


In [11]:
with Session(engine) as session:
    hero_no_team = session.exec(select(Hero).where(Hero.team_id == None)).first()
    print(hero_no_team)
    hero_no_team.team_id = 2

    session.add(hero_no_team)
    session.commit()
    session.refresh(hero_no_team)
    print("Updated hero:", hero_no_team)

secret_name='Pedro Parqueador' name='Spider-Boy' team_id=None age=None id=3
Updated hero: secret_name='Pedro Parqueador' name='Spider-Boy' team_id=2 age=None id=3
